# Platon Light Backtesting Example

This notebook demonstrates how to use the Platon Light backtesting module to develop, test, and optimize trading strategies.

## 1. Setup and Configuration

First, let's import the necessary modules and set up our configuration.

In [ ]:
import os
import sys
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pathlib import Path

# Add project root to path
sys.path.append(str(Path.cwd().parent))

# Import Platon Light modules
from platon_light.backtesting.data_loader import DataLoader
from platon_light.backtesting.backtest_engine import BacktestEngine
from platon_light.backtesting.performance_analyzer import PerformanceAnalyzer
from platon_light.backtesting.visualization import BacktestVisualizer
from platon_light.backtesting.optimization import StrategyOptimizer
from platon_light.utils.config_manager import ConfigManager

# Set up plotting
plt.style.use('seaborn-darkgrid')
sns.set_theme(style="darkgrid")
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

### Load Configuration

Let's load our backtesting configuration from the `backtest_config.yaml` file.

In [ ]:
# Load configuration
config_path = Path.cwd().parent / 'backtest_config.yaml'

if not config_path.exists():
    # If backtest_config.yaml doesn't exist, use the example config
    config_path = Path.cwd().parent / 'backtest_config.example.yaml'
    
config_manager = ConfigManager(config_path)
config = config_manager.get_config()

# Display configuration
print("Backtesting Configuration:")
print(yaml.dump(config, default_flow_style=False))

## 2. Loading Historical Data

Now, let's load historical market data for our backtesting.

In [ ]:
# Initialize data loader
data_loader = DataLoader(config)

# Define data parameters
symbol = "BTCUSDT"
timeframe = "1h"
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 3, 31)

# Load data
data = data_loader.load_data(symbol, timeframe, start_date, end_date)

# Display data
print(f"Loaded {len(data)} candles for {symbol} ({timeframe}) from {start_date} to {end_date}")
data.head()

### Visualize the Data

Let's visualize the price data to get a better understanding of the market conditions during our backtest period.

In [ ]:
# Convert timestamp to datetime
data['datetime'] = pd.to_datetime(data['timestamp'], unit='ms')

# Plot price chart
plt.figure(figsize=(14, 7))
plt.plot(data['datetime'], data['close'], label='Close Price')
plt.title(f"{symbol} Price Chart ({timeframe})", fontsize=14)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price (USDT)', fontsize=12)
plt.grid(True)
plt.legend()
plt.show()

## 3. Running a Basic Backtest

Now, let's run a basic backtest using our default strategy.

In [ ]:
# Initialize backtest engine
backtest_engine = BacktestEngine(config)

# Run backtest
results = backtest_engine.run(symbol, timeframe, start_date, end_date)

# Display basic results
print(f"Total Trades: {len(results['trades'])}")
print(f"Initial Capital: ${config['backtesting']['initial_capital']}")
print(f"Final Equity: ${results['equity_curve'][-1]['equity']:.2f}")
print(f"Return: {(results['equity_curve'][-1]['equity'] / config['backtesting']['initial_capital'] - 1) * 100:.2f}%")

## 4. Analyzing Backtest Results

Let's analyze the backtest results in more detail using the `PerformanceAnalyzer`.

In [ ]:
# Initialize performance analyzer
performance_analyzer = PerformanceAnalyzer(config)

# Analyze results
analysis = performance_analyzer.analyze(results)

# Display metrics
print("Performance Metrics:")
for metric, value in analysis['metrics'].items():
    if isinstance(value, float):
        print(f"{metric}: {value:.2f}")
    else:
        print(f"{metric}: {value}")

### Trade Analysis

Let's analyze the trades in more detail.

In [ ]:
# Get trade statistics
trade_stats = performance_analyzer.get_trade_statistics(results)

# Display trade statistics
print("Trade Statistics:")
for stat, value in trade_stats.items():
    if isinstance(value, float):
        print(f"{stat}: {value:.2f}")
    else:
        print(f"{stat}: {value}")

# Create a DataFrame from trades
trades_df = pd.DataFrame(results['trades'])

# Convert timestamps to datetime
if not trades_df.empty:
    trades_df['entry_time'] = pd.to_datetime(trades_df['entry_time'])
    trades_df['exit_time'] = pd.to_datetime(trades_df['exit_time'])
    trades_df['duration'] = trades_df['exit_time'] - trades_df['entry_time']
    
    # Display trades
    trades_df.head()

## 5. Visualizing Backtest Results

Let's visualize the backtest results using the `BacktestVisualizer`.

In [ ]:
# Initialize visualizer
visualizer = BacktestVisualizer(config)

# Plot equity curve
visualizer.plot_equity_curve(results, save=False)

In [ ]:
# Plot drawdown
visualizer.plot_drawdown(results, save=False)

In [ ]:
# Plot trade distribution
visualizer.plot_trade_distribution(results, save=False)

In [ ]:
# Plot monthly returns
visualizer.plot_monthly_returns(results, save=False)

## 6. Strategy Optimization

Now, let's optimize our strategy parameters to improve performance.

### Grid Search Optimization

First, let's use grid search to find the optimal parameter values.

In [ ]:
# Initialize optimizer
optimizer = StrategyOptimizer(config)

# Define parameter grid
param_grid = {
    "rsi_period": [7, 14, 21],
    "rsi_oversold": [20, 25, 30],
    "rsi_overbought": [70, 75, 80]
}

# Run grid search (this may take some time)
grid_results = optimizer.grid_search(
    param_grid, 
    symbol, 
    timeframe, 
    start_date, 
    end_date
)

# Display best parameters
print("Grid Search Results:")
print(f"Best Parameters: {grid_results['best_params']}")
print(f"Best Return: {grid_results['best_metrics']['return_percent']:.2f}%")
print(f"Best Sharpe Ratio: {grid_results['best_metrics']['sharpe_ratio']:.2f}")

### Visualize Optimization Results

Let's visualize the optimization results to better understand the parameter space.

In [ ]:
# Create a DataFrame from all results
results_df = pd.DataFrame(grid_results['all_results'])

# Pivot table for heatmap
if 'rsi_period' in param_grid and 'rsi_oversold' in param_grid:
    pivot = results_df.pivot_table(
        index='rsi_period', 
        columns='rsi_oversold', 
        values='return_percent'
    )
    
    # Plot heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(pivot, annot=True, cmap='viridis', fmt='.2f')
    plt.title('Return (%) by RSI Period and Oversold Level')
    plt.show()

### Run Backtest with Optimized Parameters

Now, let's run a backtest with the optimized parameters.

In [ ]:
# Update config with optimized parameters
for param, value in grid_results['best_params'].items():
    config['strategy'][param] = value
    
# Initialize backtest engine with updated config
optimized_backtest_engine = BacktestEngine(config)

# Run backtest with optimized parameters
optimized_results = optimized_backtest_engine.run(symbol, timeframe, start_date, end_date)

# Analyze optimized results
optimized_analysis = performance_analyzer.analyze(optimized_results)

# Display metrics
print("Optimized Performance Metrics:")
for metric, value in optimized_analysis['metrics'].items():
    if isinstance(value, float):
        print(f"{metric}: {value:.2f}")
    else:
        print(f"{metric}: {value}")

## 7. Comparing Strategies

Let's compare the original strategy with the optimized strategy.

In [ ]:
# Compare strategies
comparison = performance_analyzer.compare_strategies(
    [results, optimized_results], 
    ["Original Strategy", "Optimized Strategy"]
)

# Display comparison
print("Strategy Comparison:")
for metric, values in comparison['metrics'].items():
    print(f"{metric}:")
    for i, strategy in enumerate(["Original", "Optimized"]):
        if isinstance(values[i], float):
            print(f"  {strategy}: {values[i]:.2f}")
        else:
            print(f"  {strategy}: {values[i]}")

In [ ]:
# Plot strategy comparison
visualizer.plot_strategy_comparison(
    [results, optimized_results], 
    ["Original Strategy", "Optimized Strategy"],
    save=False
)

## 8. Walk-Forward Optimization

Let's perform walk-forward optimization to test the robustness of our strategy.

In [ ]:
# Run walk-forward optimization (this may take some time)
wfo_results = optimizer.walk_forward_optimization(
    param_grid,
    symbol,
    timeframe,
    start_date,
    end_date,
    window_size=30,  # 30 days in-sample window
    step_size=7      # 7 days out-of-sample window
)

# Display walk-forward optimization results
print("Walk-Forward Optimization Results:")
for i, window in enumerate(wfo_results['windows']):
    print(f"\nWindow {i+1}:")
    print(f"  In-Sample Period: {window['in_sample']['start']} to {window['in_sample']['end']}")
    print(f"  Out-of-Sample Period: {window['out_sample']['start']} to {window['out_sample']['end']}")
    print(f"  Best Parameters: {window['in_sample']['best_params']}")
    print(f"  In-Sample Return: {window['in_sample']['metrics']['return_percent']:.2f}%")
    print(f"  Out-of-Sample Return: {window['out_sample']['metrics']['return_percent']:.2f}%")

### Visualize Walk-Forward Optimization Results

In [ ]:
# Extract data for visualization
windows = []
in_sample_returns = []
out_sample_returns = []
parameter_stability = {}

for i, window in enumerate(wfo_results['windows']):
    windows.append(i+1)
    in_sample_returns.append(window['in_sample']['metrics']['return_percent'])
    out_sample_returns.append(window['out_sample']['metrics']['return_percent'])
    
    # Track parameter stability
    for param, value in window['in_sample']['best_params'].items():
        if param not in parameter_stability:
            parameter_stability[param] = []
        parameter_stability[param].append(value)

# Plot in-sample vs out-of-sample returns
plt.figure(figsize=(12, 6))
plt.bar(np.array(windows) - 0.2, in_sample_returns, width=0.4, label='In-Sample Return (%)')
plt.bar(np.array(windows) + 0.2, out_sample_returns, width=0.4, label='Out-of-Sample Return (%)')
plt.axhline(y=0, color='r', linestyle='-', alpha=0.3)
plt.xlabel('Window')
plt.ylabel('Return (%)')
plt.title('Walk-Forward Optimization: In-Sample vs Out-of-Sample Returns')
plt.xticks(windows)
plt.legend()
plt.grid(True)
plt.show()

# Plot parameter stability
for param, values in parameter_stability.items():
    plt.figure(figsize=(12, 4))
    plt.plot(windows, values, marker='o', label=param)
    plt.xlabel('Window')
    plt.ylabel('Parameter Value')
    plt.title(f'Parameter Stability: {param}')
    plt.xticks(windows)
    plt.grid(True)
    plt.legend()
    plt.show()

## 9. Multi-Asset Backtesting

Let's run backtests on multiple assets to test the strategy's robustness.

In [ ]:
# Define symbols
symbols = ["BTCUSDT", "ETHUSDT", "SOLUSDT"]

# Run backtest for each symbol
multi_asset_results = {}
for sym in symbols:
    print(f"Running backtest for {sym}...")
    results = backtest_engine.run(sym, timeframe, start_date, end_date)
    multi_asset_results[sym] = results
    
    # Display basic results
    print(f"  Total Trades: {len(results['trades'])}")
    print(f"  Return: {(results['equity_curve'][-1]['equity'] / config['backtesting']['initial_capital'] - 1) * 100:.2f}%")
    print()

### Compare Multi-Asset Results

In [ ]:
# Analyze multi-asset results
multi_asset_analysis = {}
for sym, results in multi_asset_results.items():
    multi_asset_analysis[sym] = performance_analyzer.analyze(results)

# Create comparison table
comparison_data = []
for sym, analysis in multi_asset_analysis.items():
    metrics = analysis['metrics']
    comparison_data.append({
        'Symbol': sym,
        'Return (%)': metrics['return_percent'],
        'Sharpe Ratio': metrics['sharpe_ratio'],
        'Max Drawdown (%)': metrics['max_drawdown_percent'],
        'Win Rate (%)': metrics['win_rate'],
        'Profit Factor': metrics['profit_factor'],
        'Total Trades': metrics['total_trades']
    })

# Create DataFrame
comparison_df = pd.DataFrame(comparison_data)
comparison_df.set_index('Symbol', inplace=True)

# Display comparison
comparison_df

### Plot Multi-Asset Equity Curves

In [ ]:
# Plot equity curves for all assets
plt.figure(figsize=(14, 7))

for sym, results in multi_asset_results.items():
    # Create DataFrame from equity curve
    equity_df = pd.DataFrame(results['equity_curve'])
    equity_df['datetime'] = pd.to_datetime(equity_df['timestamp'], unit='ms')
    
    # Normalize equity to percentage return
    initial_equity = config['backtesting']['initial_capital']
    equity_df['return_pct'] = (equity_df['equity'] / initial_equity - 1) * 100
    
    # Plot equity curve
    plt.plot(equity_df['datetime'], equity_df['return_pct'], label=sym)

plt.axhline(y=0, color='r', linestyle='-', alpha=0.3)
plt.title('Multi-Asset Equity Curves', fontsize=14)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Return (%)', fontsize=12)
plt.grid(True)
plt.legend()
plt.show()

## 10. Creating a Complete Backtest Report

Finally, let's create a complete HTML report with all the backtest results.

In [ ]:
# Create report for the optimized strategy
report_path = visualizer.create_report(optimized_results)
print(f"Report generated at: {report_path}")

## Conclusion

In this notebook, we've demonstrated how to use the Platon Light backtesting module to:

1. Load historical market data
2. Run backtests with different strategies and parameters
3. Analyze and visualize backtest results
4. Optimize strategy parameters using grid search and walk-forward optimization
5. Compare strategy performance across multiple assets
6. Generate comprehensive backtest reports

This provides a solid foundation for developing and testing trading strategies before deploying them in a live trading environment.